### Import

In [1]:
import pandas as pd
import ast


### 1. Caricamento del dataset

In [2]:
file_path = "./data/monsters.csv"
try:
    df = pd.read_csv(file_path)
    print(f"✅ Dataset caricato: {file_path}")
except FileNotFoundError:
    file_path = "monsters.csv"
    df = pd.read_csv(file_path)
    print(f"✅ Dataset caricato (root): {file_path}")


✅ Dataset caricato: ./data/monsters.csv


### 2. Pulizia Challenge Rating (Target)
# Manteniamo solo 'cr' (float) ed eliminiamo 'challenge_rating' (stringa "1/4").

In [3]:
if 'challenge_rating' in df.columns:
    df = df.drop(columns=['challenge_rating'])
    print("✅ Colonna 'challenge_rating' rimossa (target numerico 'cr' mantenuto).")
else:
    print("⚠️ Colonna 'challenge_rating' non trovata (già rimossa?).")


✅ Colonna 'challenge_rating' rimossa (target numerico 'cr' mantenuto).


### 3. Rimozione Legendary Desc
# Colonna puramente descrittiva, ridondante per il ML.

In [4]:
if 'legendary_desc' in df.columns:
    df = df.drop(columns=['legendary_desc'])
    print("✅ Colonna 'legendary_desc' rimossa.")


✅ Colonna 'legendary_desc' rimossa.


### 4. Action Economy & Abilities (Feature Engineering)
# Convertiamo le liste di dizionari (stringhe) in conteggi numerici.
# Esempio: "['Multiattack', 'Claw']" -> 2

In [5]:
print("--- Elaborazione Action Economy (Counting) ---")

complex_cols = [
    'actions', 
    'reactions', 
    'legendary_actions', 
    'special_abilities', 
    'spell_list'
]

def count_complex_entries(text):
    """
    Parsa la stringa come lista Python e ne conta gli elementi.
    Se il parsing fallisce, usa un fallback euristico.
    """
    if pd.isna(text) or str(text).strip() == '':
        return 0
    try:
        # Tenta di valutare la stringa come struttura Python reale
        evaluated = ast.literal_eval(str(text))
        if isinstance(evaluated, list):
            return len(evaluated)
        return 0
    except (ValueError, SyntaxError):
        # Fallback: conta le occorrenze di 'name' (chiave tipica dei JSON in questo dataset)
        return str(text).count("'name':")

for col in complex_cols:
    if col in df.columns:
        # 1. Calcolo conteggio
        counts = df[col].apply(count_complex_entries).astype(int)
        
        # 2. Inserimento nuova colonna numerica (con suffisso _count)
        # La inseriamo subito dopo la colonna originale per ordine visivo
        idx = df.columns.get_loc(col)
        new_col_name = f"{col}_count"
        
        if new_col_name not in df.columns:
            df.insert(idx + 1, new_col_name, counts)
        
        # 3. Rimozione colonna testuale originale
        df = df.drop(columns=[col])
        print(f"✅ {col} -> Convertita in {new_col_name} (Max: {counts.max()})")
    else:
        print(f"⚠️ Colonna '{col}' non trovata.")


--- Elaborazione Action Economy (Counting) ---
✅ actions -> Convertita in actions_count (Max: 11)
✅ reactions -> Convertita in reactions_count (Max: 6)
✅ legendary_actions -> Convertita in legendary_actions_count (Max: 8)
✅ special_abilities -> Convertita in special_abilities_count (Max: 13)
✅ spell_list -> Convertita in spell_list_count (Max: 26)


### 5. Salvataggio

In [6]:
try:
    df.to_csv(file_path, index=False)
    print(f"\n💾 File aggiornato e salvato in: {file_path}")
    
    # Preview finale delle colonne numeriche create
    cols_preview = ['name', 'cr'] + [f"{c}_count" for c in complex_cols if f"{c}_count" in df.columns]
    print(df[cols_preview].head())
    
except PermissionError:
    print(f"\n❌ ERRORE: Il file '{file_path}' è aperto. Chiudilo e riprova.")



💾 File aggiornato e salvato in: ./data/monsters.csv
                 name     cr  actions_count  reactions_count  \
0             Aboleth  10.00              4                0   
1             Acolyte   0.25              1                0   
2  Adult Black Dragon  14.00              6                0   
3   Adult Blue Dragon  16.00              6                0   
4  Adult Brass Dragon  13.00              6                0   

   legendary_actions_count  special_abilities_count  spell_list_count  
0                        3                        3                 0  
1                        0                        1                 6  
2                        3                        2                 0  
3                        3                        1                 0  
4                        3                        1                 0  
